In [1]:
import requests
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

## Synonyms with Thesaurus API

In [2]:
def get_synonyms_api(key_word):
    url = "https://www.dictionaryapi.com/api/v3/references/thesaurus/json/"
    key= "?key=bc125025-b3b8-4b13-b8e8-c1ef37845bad"
    syn_json = requests.get(url+key_word.lower()+key).json()
    syn_list = syn_json[0]['meta']['syns']
    if len(syn_list) > 1:
        master = sum(syn_list, [])
    elif len(syn_list) == 1:
        master = syn_list[0]
    master.append(key_word)
    return master

### Problems with some words

consumer discretionary, consumer staples, communication services (DO NOT WORK)

    -> they do not exist within the Thesaurus, therefore the JSON returns a list with possible words
Definition of Consumer Discretionary:
- goods that are non-essential but desirable if their income is sufficient to purchase them

Definition of Consumer Staples:
- goods that are essential

Definition of Communication Services:
- elecommunications Services, Cable Services, Video Services, or Information Services

-> Solution for now: remove those words from here

In [3]:
df = pd.read_csv("master_taxonomy.csv", index_col=0)#, on_bad_lines='skip')
df = df.reset_index(drop=True)
df.columns

Index(['Authors', 'Title', 'DOI', 'Link', 'Abstract', 'Author Keywords',
       'Index Keywords'],
      dtype='object')

In [4]:
df['Abstract'] = df['Abstract'].str.lower()
df['Title'] = df['Title'].str.lower()
df['Author Keywords'] = df['Author Keywords'].str.lower()
df['Index Keywords'] = df['Index Keywords'].str.lower()

In [5]:
df

,Authors,Title,DOI,Link,Abstract,Author Keywords,Index Keywords
0,"Abadía J.J.P., Fritz H., Dadoulis G., Dragos K...",automated decision making in structural health...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,the need for processing large amounts of data ...,NaN,artificial intelligence; damage detection; dec...
1,"Abbass H.A., Hunjet R.A.",smart shepherding: towards transparent artific...,10.1007/978-3-030-60898-9_1,https://www.scopus.com/inward/record.uri?eid=2...,the aim of this chapter is to uncover the beau...,explainable artificial intelligence; interpret...,NaN
2,"Abdollahi A., Pradhan B.",urban vegetation mapping from aerial imagery u...,10.3390/s21144738,https://www.scopus.com/inward/record.uri?eid=2...,urban vegetation mapping is critical in many a...,deep neural network; remote sensing; shap; veg...,aerial photography; antennas; biodiversity; de...
3,"Abdul A., Von Der Weth C., Kankanhalli M., Lim...",cogam: measuring and moderating cognitive load...,10.1145/3313831.3376615,https://www.scopus.com/inward/record.uri?eid=2...,interpretable machine learning models trade -o...,cognitive load; explainable artificial intelli...,computation theory; economic and social effect...
4,"Abe T., Furukawa R., Iwasaki Y., Ikemura T.",time-series trend of pandemic sars-cov-2 varia...,10.5334/dsj-2021-029,https://www.scopus.com/inward/record.uri?eid=2...,to confront the global threat of coronavirus d...,batch-learning self-organizing map (blsom); co...,conformal mapping; diseases; genes; machine le...
...,...,...,...,...,...,...,...
4586,[No author name available],icmlsc 2021 - proceedings of the 2021 5th inte...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 32 papers. the topics ...,NaN,NaN
4587,[No author name available],2021 ieee 29th international conference on net...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 75 papers. the topics ...,NaN,NaN
4588,[No author name available],10th international conference on computational...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 33 papers. the special...,NaN,NaN
4589,[No author name available],proceedings of the 2nd international conferenc...,NaN,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 21 papers. the topics ...,NaN,NaN


In [6]:
df = df.fillna("blank")

In [7]:
non_alp = [';', ',', '(', ')', '[', ']', '.']
for i in non_alp:
    for col in ['Title','Abstract', 'Author Keywords', 'Index Keywords']:
        df[col] = df[col].map(lambda x: x.replace(i,''))

In [8]:
df['content_sum'] = df['Abstract'] + df['Title'] + df['Author Keywords'] + df['Index Keywords']
df["content_sum"] = df["content_sum"].str.lower()
stop = stopwords.words('english')
df["stripped_content"] = df['content_sum'].apply(lambda x: " ".join([word for word in x.split() if word not in (stop)]))

In [9]:
keys = ["energy", "materials", "industrial", "financial",  "utilities"]
no_keys = [ "consumer discretionary", "consumer staples", "information technology", "communication services", "healthcare", "real estate"]

In [10]:
# Saving into a dictionary all of the syns
keys_dict_simple = {}
for i in keys:
    keys_dict_simple[i] = get_synonyms_api(i)
 
#keys_dict

In [11]:
keys_dict = {}
keys_dict["consumer discretionary"] = ["consumer discretionary","non-essential goods", "non essential goods", "non-essential products", "non essential products", "unrestricted goods", "unrestricted products", "nonobligatory goods", "nonobligatory products"]
keys_dict["consumer staples"] = ["essential goods", "essential goods", "essential products", "essential products", "restricted goods", "restricted products", "obligatory goods", "obligatory products"]
keys_dict["information technology"] = ["IT", "information techonology", "i.t", "communication science", "communication sciences", "computer science", "info tech", "cybernetics"]
keys_dict["communication services"] = ["call service", "communication department",  "communication facilities", "communication service","communications department","communications facilities", "communications service","communications services", "facilities-based", "telecommunications service","media service","reporting services", "telecom service", "telecom services", "telecommunication service","telecommunication services"]
keys_dict["healthcare"] = ["medical care","healthcare","medical treatment","medical assistance","medical attention","medical aid","health protection","health maintenance","medical attendance","medical help","medical management","medical service","preventive medicine","wellness program","health management","curative care","public health","maintenance of health","health care service","primary care","medical services","care insurance","health insurance","medical coverage","medical insurance"]
keys_dict["real estate"] = ["building","buildings","housing","immovable","land","motionless","premises","Properties","Property","realtor"]

In [12]:
for key in keys_dict_simple.keys():
    df[key] = 0
    print(key)
    for phr in keys_dict_simple[key]:
        #print(phr)
        for i in range(len(df)):
            x = df['stripped_content'].values[i].split()
            if phr in (x):
                df[key][i] += 1
    #df.loc[key, i] = count
                
#df

energy


<ipython-input-12-6e1249b3cb3b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key][i] += 1


materials


<ipython-input-12-6e1249b3cb3b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key][i] += 1


industrial


<ipython-input-12-6e1249b3cb3b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key][i] += 1


financial


<ipython-input-12-6e1249b3cb3b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key][i] += 1


utilities


<ipython-input-12-6e1249b3cb3b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key][i] += 1


In [15]:
df[df[""] > 1]

,Authors,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,content_sum,stripped_content,energy,materials,industrial,financial,utilities
117,"Bai Z., Ravi S.S., Davidson I.",towards description of block model on graph,10.1007/978-3-030-67664-3_3,https://www.scopus.com/inward/record.uri?eid=2...,existing block modeling methods can detect com...,block model explainable artificial intelligenc...,combinatorial optimization data mining inducti...,existing block modeling methods can detect com...,existing block modeling methods detect communi...,2,0,1,0,0
125,"Baniecki H., Kretowicz W., Piatyszek P., Wisni...",dalex: responsible machine learning with inter...,blank,https://www.scopus.com/inward/record.uri?eid=2...,in modern machine learning we observe the phen...,explainability fairness interactivity interpre...,laws and legislation machine learning python b...,in modern machine learning we observe the phen...,modern machine learning observe phenomenon opa...,2,0,1,0,0
127,"Bao F., Deng Y., Du M., Ren Z., Wan S., Liang ...",explaining the genetic causality for complex p...,10.1016/j.patter.2020.100057,https://www.scopus.com/inward/record.uri?eid=2...,the genetic effect explains the causality from...,association analysis deep learning disease cau...,diseases genes learning systems association an...,the genetic effect explains the causality from...,genetic effect explains causality genetic muta...,2,0,0,0,0
140,"Belle V., Papantonis I.",principles and practice of explainable machine...,10.3389/fdata.2021.688969,https://www.scopus.com/inward/record.uri?eid=2...,artificial intelligence ai provides many oppor...,black-box models explainable ai machine learni...,blank,artificial intelligence ai provides many oppor...,artificial intelligence ai provides many oppor...,3,0,1,0,0
184,"Braun M., Kellner L., Schreiber S., Ehlers S.",prediction of fatigue failure in small-scale b...,10.1016/j.prostr.2022.03.019,https://www.scopus.com/inward/record.uri?eid=2...,butt-welded joints are common in many industri...,explainable ai fatigue life prediction fatigue...,blank,butt-welded joints are common in many industri...,butt-welded joints common many industries fati...,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,"Weller O., Sagers L., Hanson C., Barnes M., Sn...",predicting suicidal thoughts and behavior amon...,10.1371/journal.pone.0258535,https://www.scopus.com/inward/record.uri?eid=2...,introduction addressing the problem of suicida...,blank,adolescent adult algorithm article controlled ...,introduction addressing the problem of suicida...,introduction addressing problem suicidal thoug...,3,0,0,0,1
4534,"Woody S., Carvalho C.M., Murray J.S.",model interpretation through lower-dimensional...,10.1080/10618600.2020.1796684,https://www.scopus.com/inward/record.uri?eid=2...,nonparametric regression models have recently ...,decision theory graphical summary interpretabl...,blank,nonparametric regression models have recently ...,nonparametric regression models recently surge...,2,1,0,0,1
4537,"Xie F., Ong M.E.H., Liew J.N.M.H., Tan K.B.K.,...",development and assessment of an interpretable...,10.1001/jamanetworkopen.2021.18467,https://www.scopus.com/inward/record.uri?eid=2...,importance: triage in the emergency department...,blank,adult article breathing rate cardiac arrest ri...,importance: triage in the emergency department...,importance: triage emergency department ed com...,2,0,0,0,1
4541,"Xu S.G., Reich B.J.",bayesian nonparametric quantile process regres...,10.1111/biom.13576,https://www.scopus.com/inward/record.uri?eid=2...,flexible estimation of multiple conditional qu...,density regression hamiltonian monte carlo int...,blank,flexible estimation of multiple conditional qu...,flexible estimation multiple conditional quant...,2,0,0,0,0


In [ ]:
x = df["content_sum"].values[1].split()
x#1]

In [ ]:
count1 = 0
count2 = 1

master_list = []

while count2 < (len(df) - 1):
    val = df["content_sum"].values[count1].split()
    for i in range(len(val)):
        print(count1)
        x = val[count1]
        y = val[count2]
        master_list.append(x + " " + y) #+ " " + val[count2])
        count1 += 1
        count2 += 1

In [ ]:
for m in range(len(df)):
    df['master_content_sum'] = " "
    for i in df['content_sum'].values[m].split():
        for j in df['content_sum'].values[m].split():
            df.iloc[m, -1] = [i + " " + j]
            #df['master_content_sum'][m].append()

In [ ]:
df

In [ ]:
for key in key_words:
    df[key] = 0
    for phr in get_synonyms(key):
        for i in range(len(df)): 
            if phr in (df['content_sum'].values[i].split()):
                df[key][i] += 1
    #df.loc[key, i] = count